#### Важное требование ко всей домашке в целом: в jupyter ноутбуке не должно был лишнего кода (т.е. если вы взяли за основу семинар, не забудьте удалить все лишнее)

In [ ]:
!pip install pymorphy2
!pip install razdel

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

from IPython.display import Image
from IPython.core.display import HTML 
%matplotlib inline

from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import gensim

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window не так используется)  
3) измените часть с np.random.randint(vocab_size) так, чтобы случайные негативные примеры выбирались обратно пропорционально частотностям слов (частотные должны выбираться реже, а редкие чаще)

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

In [3]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

In [4]:
texts = open('wiki_data.txt').read().splitlines()[:5000]

In [5]:
vocab = Counter()

wiki = texts 
for text in wiki:
    vocab.update(preprocess(text))

filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

In [6]:
word2id = { 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [7]:
id2word = {i:word for word, i in word2id.items()}

In [8]:
sentences = []

for text in wiki:
    tokens = preprocess(text)
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

In [11]:
# skip gram
def gen_batches_sg(sentences, window = 6, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window//2):i] + sent[i+1:i+window//2]
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)
                    
                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size)) #не знаю как ((
                    y.append(0)
                    
                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []


In [12]:
# # cbow 
def gen_batches_cbow(sentences, window = 12, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window//2):i] + sent[i+1:i+window//2]

                X_target.append(word)
                X_context.append(context)
                y.append(1)
                
                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window*2)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [13]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(1,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
vocab_size = len(id2word)
model.fit(gen_batches_sg(sentences[:1000], window=6),
          validation_data=gen_batches_sg(sentences[1000:],  window=6),
          batch_size=1000,
          steps_per_epoch=1000,
          validation_steps=30,
          epochs=2)  

In [17]:
embeddings = model.layers[2].get_weights()[0]

In [18]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in 
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [19]:
most_similar('церковь', embeddings)

['церковь',
 'православный',
 'приход',
 'храм',
 'синод',
 'епархия',
 'здание',
 'икона',
 'монастырь',
 'рпц']

In [25]:
most_similar('собака', embeddings)

['собака',
 'энергия',
 'поведение',
 'воздействие',
 'снижение',
 'богиня',
 'пломба',
 'приобретать',
 'животное',
 'смысл']

In [26]:
#cbow negative sampling
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

# полученную близость нужно преобразовать в вероятность
# когда она одна используется не софтмакс и сигмоида
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(gen_batches_cbow(sentences[:1000], window=12),
          validation_data=gen_batches_cbow(sentences[1000:],  window=12),
          batch_size=1000,
          steps_per_epoch=1000,
          validation_steps=30,
          epochs=2)

In [29]:
embeddings = model.layers[2].get_weights()[0]

In [30]:
most_similar('церковь', embeddings)

['церковь',
 'здание',
 'храм',
 'православный',
 'собор',
 'приход',
 'часовня',
 'освятить',
 'икона',
 'святой']

In [32]:
most_similar('собака', embeddings)

['собака',
 'поведение',
 'матрица',
 'шрифт',
 'полагать',
 'коэффициент',
 'богиня',
 'координата',
 'последовательность',
 'ухо']

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [33]:
from gensim.models import Word2Vec

In [51]:
texts_2 = open('wiki_data.txt').read().splitlines()[:10000] #возьму побольше, а то пишет, что таких слов нет в словаре

In [52]:
texts_2 = [preprocess(text) for text in texts_2]

In [60]:
#у меня не работает epochs, got an unexpected argument
w2v = gensim.models.Word2Vec(texts_2,
                             size=150, 
                             min_count=100, 
                             max_vocab_size=10000,
                             window=6,
                             negative = 20,
                             sg=1,
                             hs=1,
                             ns_exponent=1.0,
                             sample=1e-4)

In [61]:
w2v.wv.most_similar('церковь')

[('православный', 0.7796241044998169),
 ('храм', 0.7412840127944946),
 ('монастырь', 0.7142841219902039),
 ('собор', 0.7101009488105774),
 ('приход', 0.6689189672470093),
 ('часовня', 0.6463629007339478),
 ('церковный', 0.6461930274963379),
 ('деревянный', 0.634380042552948),
 ('епархия', 0.6321039199829102),
 ('святой', 0.6302167773246765)]

In [62]:
w2v.wv.most_similar('кладбище')

[('похоронный', 0.7264834642410278),
 ('могила', 0.6168270111083984),
 ('скончаться', 0.4957941770553589),
 ('москва', 0.4776769280433655),
 ('часовня', 0.4676167964935303),
 ('собор', 0.4338022768497467),
 ('умереть', 0.4176018536090851),
 ('храм', 0.4157324731349945),
 ('северо-западный', 0.41288867592811584),
 ('старое', 0.41215887665748596)]

In [64]:
ft = gensim.models.FastText(texts_2,
                            min_n=4,
                            max_n=7)

In [65]:
ft.wv.most_similar('церковь')

[('«церковь', 0.9992543458938599),
 ('церкви»', 0.9651113152503967),
 ('монастырь', 0.9380764961242676),
 ('собор', 0.9261490106582642),
 ('храм', 0.9037321209907532),
 ('штырь', 0.9020834565162659),
 ('церн', 0.8665990829467773),
 ('берсерков', 0.8647642135620117),
 ('монархия', 0.8552401661872864),
 ('архия', 0.8538483381271362)]

In [66]:
ft.wv.most_similar('кино')

[('кино»', 0.9890587329864502),
 ('киноактёр', 0.9598212838172913),
 ('кинозал', 0.9594863653182983),
 ('кинорежиссёр', 0.942828357219696),
 ('киновед', 0.9426568150520325),
 ('кинотеатр', 0.9348472356796265),
 ('драма', 0.9315077662467957),
 ('спектакль', 0.9257732629776001),
 ('актёр', 0.9239763021469116),
 ('киноэкран', 0.920710563659668)]

# Задание 3 (4 балла)

Используя датасет для классификации (labeled.csv) и простую нейронную сеть (последняя модель в семинаре), оцените качество полученных эмбедингов в задании 1 и 2 (4 набора эмбедингов), также проверьте 1 любую из предобученных моделей с rus-vectores (но только не tayga_upos_skipgram_300_2_2019). 
Какая модель показывает наилучший результат?

Убедитесь, что для каждой модели вы корректно воспроизводите пайплайн предобработки (в 1 задании у вас лемматизация, не забудьте ее применить к датасету для классификации; у выбранной предобученной модели может быть своя специфичная предобработка - ее нужно воспроизвести)

In [67]:
data = pd.read_csv('labeled.csv')
data['norm_text'] = data.comment.apply(preprocess)

In [68]:
data.head()

,comment,toxic,norm_text
0,"Верблюдов-то за что? Дебилы, бл...\r\n",1.0,"[верблюд-то, за, что, дебил, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохол, это, отдушина, затюканый, россиянин, м..."
2,Собаке - собачья смерть\r\n,1.0,"[собака, , собачий, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страница, обновить, дебил, это, тоже, не, оск..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[ты, не, убедить, 6-страничный, пдф, в, тот, ч..."


In [ ]:
vocab = Counter()

for text in data['norm_text']:
    vocab.update(text)
    
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 5:
        filtered_vocab.add(word)

len(filtered_vocab)

In [70]:
word2id = { 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
id2word = {i:word for word, i in word2id.items()}

In [71]:
X = []

for tokens in data['norm_text']:
    ids = [word2id[token] for token in tokens if token in word2id]
    X.append(ids)

In [72]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=100)
y = data.toxic.values

In [73]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [ ]:
#ft = gensim.models.FastText(texts + data['norm_text'].values.tolist(), window=10, epochs=10)

In [74]:
weights = np.zeros((len(word2id), 100))

for word, i in word2id.items():
    # вектор паддинга оставим нулевым
    if word == 'PAD':
        continue

    try:
        weights[i] = ft.wv[word]
    
    
    except KeyError:
        # для слов, которых нет в модели тоже возьмем какой-то  рандомный вектор
        continue
        weights[i] = ft.wv['опрагпллирао']

In [75]:

inputs = tf.keras.layers.Input(shape=(100,))

# передаем матрицу в эмбединг слой
# указываем параметр trainable=False, чтобы вектора не обучались
# Пояснение: кажется, что обучение векторов отключать не стоит, так как модель должна взять готовые 
# эмбединги и подогнать их к нашей задаче. На практитке это однако не работает - эмбединги просто 
# обучатся заново
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100, 
                                       trainable=False,
                                       weights=[weights])(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
          epochs=30)

In [79]:
#сложно(( не понимаю